In [262]:
from seek_scraper import SeekScraper
from data_preparation import keywords_finder, salary_processor, date_processor
import json
import pandas as pd
import os

In [263]:
# skills keywords list
with open('data/keywords/skill_keywords.json') as f:
    skills = json.load(f)
# programming languages keywords list
with open('data/keywords/programming_keywords.json') as f:
    langugages = json.load(f)

In [264]:
df = pd.read_csv('data/processed/seek_scraper_processed_data.csv', na_filter=False)
# create a new column called has valid salary
# valid salary range 50000 - 500000
df['has_valid_salary'] = df['per_annum'].apply(lambda x: "Salary Mentioned" if x >= 50000 and x <= 500000 else "Salary Not Mentioned")

In [265]:
import plotly.graph_objects as go
# show the skills, languages or qualifications
val_count  = pd.DataFrame(df['has_valid_salary'].value_counts())
# add percentage as a new column 
val_count['percentage'] = val_count['has_valid_salary'].apply(lambda x: round(x/val_count['has_valid_salary'].sum()*100,1))
# produce same graph using go
fig = go.Figure(data=[go.Bar(y=val_count.index, x=val_count['has_valid_salary'], 
                            orientation='h', text=val_count['percentage'], 
                            textposition='outside', 
                            showlegend=False, 
                            texttemplate='%{text:.2s}%', 
                            marker_color=['#ff7f0e','#1f77b4'])
                            ])
# increase the font size and make it black and bold
fig.update_layout(font=dict(size=12, color='black'))

# set the title
fig.update_layout(title_text='Job Postings with Valid Salary')

# set size to 1280 x 720
fig.update_layout(width=1280, height=720)

In [266]:
# produce a frequency distribution of the salary where the salary is mentioned
valid_sals = df[df['has_valid_salary'] == 'Salary Mentioned']
import plotly.express as px

fig = px.histogram(valid_sals, x="per_annum", nbins=100, title="Salary Distribution")
# make it 1280x720
fig.update_layout(width=1280, height=720)
fig.show()

In [267]:
# average salary by job work type
avg_sal = valid_sals.groupby('jobWorkType')['per_annum'].mean().reset_index()
avg_sal['Average Annual Salary'] = avg_sal['per_annum'].apply(lambda x: round(x, 2))
# sort per_annum in descending order
avg_sal = avg_sal.sort_values(by='per_annum', ascending=True)

fig = go.Figure(data=[go.Bar(y=avg_sal['jobWorkType'], x=avg_sal['Average Annual Salary'],
                            orientation='h', text=avg_sal['Average Annual Salary'], 
                            textposition='outside', 
                            showlegend=False, 
                            texttemplate='%{text:.2s}'
                            )
                            ])
# increase the font size and make it black and bold
fig.update_layout(font=dict(size=12, color='black'))

# set the title
fig.update_layout(title_text='Average Salary by Job Type')

# set size to 800x400
fig.update_layout(width=1280, height=720)
fig.show()

In [268]:
skill_salary = {}
for key,val in skills.items():
    count = 0
    sal = 0
    for index, row in valid_sals.iterrows():
        if val in row['Skill']:
            count += 1
            sal += row['per_annum']
    if count > 10:
        skill_salary[val] = sal/count
skill_salary = {k: v for k, v in sorted(skill_salary.items(), key=lambda item: item[1], reverse=True)}
skill_salary = pd.DataFrame.from_dict(skill_salary, orient='index', columns=['per_annum'])
skill_salary = skill_salary.reset_index()
skill_salary.columns = ['skill', 'per_annum']
skill_salary['per_annum'] = skill_salary['per_annum'].apply(lambda x: round(x, 2))
skill_salary = skill_salary.sort_values(by='per_annum', ascending=True)
# only show the top 20 skills
skill_salary = skill_salary.tail(30)
fig = go.Figure(data=[go.Bar(y=skill_salary['skill'], x=skill_salary['per_annum'],
                            orientation='h', text=skill_salary['per_annum'], 
                            textposition='outside', 
                            showlegend=False, 
                            texttemplate='%{text:.4s}'
                            )
                            ])
# increase the font size and make it black and bold
fig.update_layout(font=dict(size=12, color='black'))

# set the title
fig.update_layout(title_text='Average Salary by Skills')

# set size to 1280 x 720
fig.update_layout(width=1280, height=720)
fig.show()

In [269]:
language_salary = {}
for key,val in langugages.items():
    count = 0
    sal = 0
    for index, row in valid_sals.iterrows():
        if val in row['Programming']:
            count += 1
            sal += row['per_annum']
    if count > 10:
        language_salary[val] = sal/count
language_salary = {k: v for k, v in sorted(language_salary.items(), key=lambda item: item[1], reverse=True)}
language_salary = pd.DataFrame.from_dict(language_salary, orient='index', columns=['per_annum'])
language_salary = language_salary.reset_index()
language_salary.columns = ['language', 'per_annum']
language_salary['per_annum'] = language_salary['per_annum'].apply(lambda x: round(x, 2))
language_salary = language_salary.sort_values(by='per_annum', ascending=True)
# show only top 30
language_salary = language_salary.tail(30)
fig = go.Figure(data=[go.Bar(y=language_salary['language'], x=language_salary['per_annum'],
                            orientation='h', text=language_salary['per_annum'], 
                            textposition='outside', 
                            showlegend=False, 
                            texttemplate='%{text:.4s}'
                            )
                            ])
# increase the font size and make it black and bold
fig.update_layout(font=dict(size=12, color='black'))

# set the title
fig.update_layout(title_text='Average Salary by Programming Language')

# set size to 1280 x 720
fig.update_layout(width=1280, height=720)

In [270]:
# get rid of those where jobSalary column is Unknown
not_valid_sals = df[(df['jobSalary'] != 'Unknown') & (df['per_annum'] == 0)]
excuses = not_valid_sals['jobSalary'].value_counts()
# get from excuses where there is no alphanumeric character
excuses = excuses[excuses.index.str.contains('[a-zA-Z]')]
# get the top 25 excuses
excuses = excuses[:25]
# drop the row where the index contains 55k 
excuses = excuses.drop(excuses.index[excuses.index.str.contains('55k')])
# write excuses to csv
excuses.to_csv('excuses.csv')
# rename index as job count
excuses.index.name = 'Excuses'
# plot the excuses as a bar chart
fig = px.bar(excuses, x=excuses.index, y='jobSalary', title='Excuses for not mentioning salary')
fig.show()

In [271]:
excuses_df = pd.DataFrame(columns=["weight","word","color","url"])
excuses_df['weight'] = excuses.values
excuses_df['word'] = excuses.index
excuses_df['color'] = '#ff7f0e'
excuses_df['url'] = 'https://www.google.com/search?q=' + excuses_df['word']
excuses_df.to_csv('excuses_df.csv', index=False, sep=';')

In [272]:
# average per_annum by job title where the count is at least 5 and the job title contains engineer
eng_sal = valid_sals[valid_sals['jobTitle'].str.contains('engineer', case=False)]
sal_avg = valid_sals.groupby('jobTitle')['per_annum'].mean().reset_index()
eng_count = eng_sal.groupby('jobTitle')['per_annum'].count().reset_index()
eng_count = eng_count[eng_count['per_annum'] > 5]
sal_avg = sal_avg.merge(eng_count, on='jobTitle')
sal_avg = sal_avg.sort_values(by='per_annum_x', ascending=True)
sal_avg = sal_avg.tail(30)
fig = go.Figure(data=[go.Bar(y=sal_avg['jobTitle'], x=sal_avg['per_annum_x'],
                            orientation='h', text=sal_avg['per_annum_x'],
                            textposition='outside',
                            showlegend=False,
                            texttemplate='%{text:.4s}'
                            )
                            ])
# increase the font size and make it black and bold
fig.update_layout(font=dict(size=12, color='black'))
fig.update_layout(title_text='Average Salary range for Data engineers')
fig.update_layout(width=1280, height=720)
fig.show()


In [273]:
eng_sal = valid_sals[valid_sals['jobTitle'].str.contains('data', case=False)]
sal_avg = valid_sals.groupby('jobTitle')['per_annum'].mean().reset_index()
eng_count = eng_sal.groupby('jobTitle')['per_annum'].count().reset_index()
eng_count = eng_count[eng_count['per_annum'] < 5]
sal_avg = sal_avg[sal_avg['per_annum'] > 300000]
sal_avg = sal_avg.merge(eng_count, on='jobTitle')
sal_avg = sal_avg.sort_values(by='per_annum_x', ascending=True)
# only keep the job title to 30 characters
sal_avg['jobTitle'] = sal_avg['jobTitle'].apply(lambda x: x[:30])
sal_avg = sal_avg.tail(30)
fig = go.Figure(data=[go.Bar(y=sal_avg['jobTitle'], x=sal_avg['per_annum_x'],
                            orientation='h', text=sal_avg['per_annum_x'],
                            textposition='outside',
                            showlegend=False,
                            texttemplate='%{text:.4s}'
                            )
                            ])
# increase the font size and make it black and bold
fig.update_layout(font=dict(size=12, color='black'))
fig.update_layout(title_text='Higher Salary Range for Data Engineers')
fig.update_layout(width=1280, height=720)
fig.show()